# Nucleus Detection

### Next steps

* Work out a loss function
* Decide how to run on other image sizes
* Initialization of weights?

### Mid term plans

* Data augmentation

# Imports

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import data_loaders, prepare_submission, net, mask_prep

import matplotlib.pyplot as plt
import numpy as np
import torch

# Data

In [3]:
train_raw, train_keys_raw = data_loaders.training_images()
# test, test_keys = data_loaders.test_images()

In [4]:
use_saved_weights = True
if use_saved_weights:
    summary_masks_raw, weights_raw = data_loaders.weights()
else: # slow slow slow...
    masks = data_loaders.training_masks()
    summary_masks_raw, weights_raw = mask_prep.weights_from_all_masks(masks)
    print("You probably want to save these weights now")
    data_loaders.save_weights(summary_masks_raw, weights_raw)

In [7]:
# For now, let us reduce our input data to just things that are 256x256
train, train_keys, weights, summary_masks = [], [], [], []
for i in range(len(train_raw)):
    if train_raw[i].shape == (256, 256):
        train.append(train_raw[i])
        train_keys.append(train_keys_raw[i])
        weights.append(weights_raw[i])
        summary_masks.append(summary_masks_raw[i])
print(len(train), len(train_keys), len(weights), len(summary_masks))

334 334 334 334


In [23]:
# Convert from numpy bytes to torch Variable floas
trainV = torch.autograd.Variable(torch.from_numpy(np.array(train).astype(np.float32)))
trainV.unsqueeze_(1)
print(trainV.shape, type(trainV))

# and weights
weightsV = torch.autograd.Variable(
    torch.from_numpy(np.array(weights).astype(np.float32))
)
weightsV.unsqueeze_(1)
print(weightsV.shape, type(weightsV))

# and summary masks
summary_masksV = torch.autograd.Variable(
    torch.from_numpy(np.array(summary_masks).astype(np.float32))
)
summary_masksV.unsqueeze_(1)
print(summary_masksV.shape, type(summary_masksV))

torch.Size([334, 1, 256, 256]) <class 'torch.autograd.variable.Variable'>
torch.Size([334, 1, 256, 256]) <class 'torch.autograd.variable.Variable'>
torch.Size([334, 1, 256, 256]) <class 'torch.autograd.variable.Variable'>


# UNET

In [16]:
n = net.u_net()
optimizer = torch.optim.SGD(n.parameters(), lr=0.0001, momentum=0.9)

In [17]:
# Run something through it as a sanity check
inp = torch.autograd.Variable(torch.randn(1, 1, 348, 348))
n(inp).shape

torch.Size([1, 1, 260, 260])

In [26]:
batchsize = 1
for i in range(len(trainV)):
    # Define input data
    inp = trainV[i:i+batchsize]
    assert inp.shape == (batchsize, 1, 256, 256)
    inp = torch.nn.ReflectionPad2d((348-256)//2)(inp)
    assert inp.shape == (batchsize, 1, 348, 348)
    
    # Define expected data and weights
    weight = weightsV[i:i+batchsize]
    assert weight.shape == (batchsize, 1, 256, 256)
    exp = summary_masksV[i:i+batchsize]
    assert exp.shape == (batchsize, 1, 256, 256)
    
    # Ready net
    optimizer.zero_grad()
    
    # Run net and get output ready for loss and backgrad
    out = n(inp)
    assert out.shape == (batchsize, 1, 260, 260)
    out = out[:,:,2:-2,2:-2]
    assert out.shape == (batchsize, 1, 256, 256)
    
    # Calculate loss
    loss = (weight * (exp - out).pow(2)).sum()
    loss.backward()
    optimizer.step()
    print(loss)

Variable containing:
 3.3311e+26
[torch.FloatTensor of size 1]



RuntimeError: value cannot be converted to type double without overflow: inf